# IMPORTANT Prerequisites

API keys for third party tools are not provided. 

- For OpenAI, we are using keys from https://platform.openai.com/account/api-keys
- For Google News Search, we are using keys from https://programmablesearchengine.google.com/controlpanel/all.
- For Google Search, we are using APIs and keys available from https://console.developers.google.com/

After running the next step, double-click the file `config.env` using the Files pane, fill in your own API keys, and save the file to proceed. 

In [1]:
!git clone https://github.com/ethancjackson/tmls-2023-material.git
!cp -r ./tmls-2023-material/data .
!cp ./tmls-2023-material/requirements.txt requirements.txt
!cp ./tmls-2023-material/config.env config.env

Cloning into 'tmls-2023-material'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), 12.33 MiB | 9.84 MiB/s, done.


# Building customizable converational applications with GPT-4

At ChainML, we're building a Conversational AI Engine for composable AI Agents that emphasizes composability, safety, and production readiness. While we are getting ready to launch our open-source Engine framework, we wanted to take the opportunity to share lessons we've learned about building solutions with GPT-3.5 and GPT-4 with the TMLS community. 

## Motivating Example

Suppose that you want to use AI to answer questions about company financial reports and operations. This could be a very useful, time-saving tool to provide to financial or business analysts as they do their day-to-day work. On its own, ChatGPT can provide useful insights and information, but it (self-admittedly) has some important blind spots:

--------

**Human User**: *Who is the CEO of Loblaw?*

**ChatGPT**: *As of my knowledge cutoff in September 2021, Galen G. Weston was the CEO of Loblaw Companies Limited. However, please note that executive positions can change over time, and **it's always a good idea to verify the latest information from reliable sources**.*

--------

**Human User**: *Can you summarize Loblaw's corporate strategy?*

**ChatGPT**: *Loblaw Companies Limited is one of the largest retail companies in Canada. **While I can provide a general overview of Loblaw's corporate strategy as of my knowledge cutoff in September 2021, it's important to note that specific details may have evolved since then**.* 

*Loblaw's corporate strategy revolves around several key pillars:*

- *Retail Operations: Loblaw focuses on operating a wide range of retail formats, including grocery stores, pharmacies, apparel stores, and more. They aim to provide a diverse selection of products and services to meet the needs of their customers.*

 (...)

- *Sustainability and Social Responsibility: Loblaw places a strong emphasis on sustainability and social responsibility. They aim to reduce their environmental impact, promote sustainable sourcing, support local communities, and provide healthier food options to customers.*

*Please note that this is a general summary, and specific details may have changed since my knowledge cutoff. For the most accurate and up-to-date information, **it's advisable to refer to official sources or recent company announcements**.*

--------

If our goal is to use AI as sort of **Corporate Analyst's Assistant**, ChatGPT is sending very clear signals: while it has the ability to answer the questions, it is not able to do so confidently or verify its own claims. In this workshop, we'll explore how we're adding these capabilities to GPT models by leveraging our AI Agent abstraction. We develop a solution based on three key components:

1. Information Retrieval and Context Building
2. Self-Assessment
3. Intelligent Control

### Information Retrieval and Context Building

While it is very impressive that LLMs on their own can answer certain factual questions, they do not have access to real-world data outside of their training sets. From a user experience perspective, ChatGPT isn't shy about this and encourages its users to dig deeper on their own. Alternatively, there are many established and emerging techniques for *providing context* to GPT and other LLM-based models. Broadly, these are techniques for finding relevant text passages to provide as part of the input to an LLM call. Web and news search APIs from Google are commonly used for this purpose. 

It's also possible index specific, relevant documents for context building. Suppose that we expect the answer to a question to be found in a company's annual report. With GPT-3.5 and GPT-4, it's not currently possible to pass in the entire document as context -- this would exceed the input size limits. Instead, we can use a flexible and tunable technique for locating and prioritizing the most relevant sections (or chunks) of the document and provide only these chunks as additional context in our LLM call.

For this workshop, we'll explore context building by providing GPT-4-based Agents with two additional skills: Google News Search and Financial Reports Querying. After adding these features, we expect the conversation to look more like this: 

--------
**Human User**: *Who is the current CEO of Loblaw?*

**Agent**: *Thank you for the question, let me search Google News and look into the most recent annual report from Loblaw to help answer the question...*

--------

### Self-Assessment

Providing relevant information to LLMs like GPT-4 is crucial for enabling them to answer more complex, factual questions. LLMs, on the other hand, are prone to *hallucinate* or completely fabricate false responses that are, on the surface, often quite convincing. This is concerning for many reasons, not the least of which is the obvious potential to be used for the deliberate generation of very convincing misinformation at-scale. 

But for ethically-minded actors, like ourselves, we recognize this flaw and focus on doing what we can to build more trustworthy Agents. One way to do this is to implement self-assessment or critiquing mechanisms, like automated fact checking. For this workshop, we'll show how the concept of Evaluator Agents can be used to implement fact checking against multiple sources of information. 

Importantly, we acknowledge that exhaustive and decisive fact checking would be complex and difficult to develop -- there are entire companies devoted only to this! With that in mind, we take a simpler approach using the question: "*Have I been able to find supporting evidence for this response in my knowledge base?*"

In this workshop, we'll consider the following sources of information for self-assessment:
1. Google News
2. Google Search
3. Wikipedia

### Intelligent Control 

To build an Agent that can leverage Context Building and Self-Assessment skills, we can bring these skills together into a **Controller** module -- which is the module by which the Agent will decide what to do next and how to leverage its skills. 

As an example:

--------
**Human User**: *Who is the current CEO of Loblaw?*

**Agent**: *Thank you for the question, I'll search Google News for this using the search query "CEO of Loblaw in June 2023".*

--------

Later in the workshop, we'll show how we think about developing Controllers so that Agents can decide which skill to use next and *how* to apply it, for example by reformulating a search query.


# Workshop Overview

During the workshop, we'll explore and experiment with the following techniques:

1. Creating searchable databases of company financial documents
2. Creating an Analyst Agent that knows how and when to access this data
3. Creating an Evaluator Agent that applies fact-checking against Analyst Agents' responses
4. Creating an end-to-end solution for answering financial and corporate questions

While this workshop is focused on answering questions about company financial and operations documents, the same general approach applies to many other domains, like customer support, academic literature review, and even social media analysis. 



## Data Files

We're using the following data sources for this workshop: 
- [Loblaw Annual Reports and Information Forms](https://www.loblaw.ca/en/investors-reports/)
- [RBC Annual Reports and Information Forms](https://www.rbc.com/investor-relations/)

We have downloaded the 2022 Annual Reports and Information Forms for each of these companies, this notebook expects to find them in the `./data/` directory. 

# Technical Setup

### Install dependencies

In [2]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 53.7 MB/s eta 0:00:00
    

### Import dependencies

In [3]:
# Python Standard Library imports
from string import Template
import os
from heapq import nlargest

# Third-Party Library imports
import json
import requests
import numpy as np
from dotenv import load_dotenv
from tqdm import tqdm
from googleapiclient.discovery import build
from pprint import pprint

# Sentence Transformers imports
from sentence_transformers import SentenceTransformer, CrossEncoder

# Transformers import
from transformers import AutoTokenizer

# OpenAI import
import openai

# GoogleNews import
from GoogleNews import GoogleNews

# WikipediaAPI import
import wikipediaapi

# Tiktoken import
import tiktoken

# ChromaDB imports
import chromadb
from chromadb.config import Settings

# Load the environment variable and set OpenAI API key
load_dotenv('config.env', override=True)
openai.api_key = os.getenv("OPENAI_API_KEY")


# Making Calls to GPT-4

In [4]:
# Function for getting a single chat completion
def get_completion(message, system_prompt='', model='gpt-4'):
  completion = openai.ChatCompletion.create(
    model=model,
    temperature=0,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": message}
    ]
  )
  return completion["choices"][0]["message"]["content"]

In [5]:
user_message = "What are Loblaw's environmental priorities?"
print(get_completion(user_message, model="gpt-4"))

Loblaw Companies Limited, a Canadian food retailer, has outlined several environmental priorities as part of its corporate social responsibility strategy. Some of these priorities include:

1. Reducing greenhouse gas emissions: Loblaw is committed to reducing its carbon footprint by improving energy efficiency, investing in renewable energy, and reducing refrigerant emissions in its stores and distribution centers.

2. Reducing waste: The company aims to minimize waste generation by implementing waste reduction initiatives, such as recycling programs, food waste diversion, and reducing packaging materials.

3. Sustainable sourcing: Loblaw is focused on sourcing products responsibly by working with suppliers who adhere to sustainable practices, such as fair labor conditions, animal welfare, and responsible use of natural resources.

4. Promoting sustainable seafood: The company is committed to offering sustainable seafood options to its customers by sourcing from responsible fisheries a

# Context-Building Skills

## Data Preprocessing

A standard approach to doing text-based search involves the transformation of sequences of text (chunks) into embedding vectors. These are high-dimensional represenations of text sequences produced by a pre-trained language model. Many of these are available in the `sentence_transformers` Python package. 

After converting text chunks to embedding vectors, we'll be able to match them to a user (or AI-generated) query by computing a distance metric between each of the query-chunk embedding pairs. Even with thousands of chunks, this process can be computed almost instantly and will enable us to quickly identify parts of the source documents that are related to the user's question.

In [6]:
import hashlib

from transformers import AutoTokenizer

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
EMBEDDING_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'

embedding_tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)

def tokenizer_len(text):
    tokens = embedding_tokenizer.encode(text)
    return len(tokens) 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=64,
    length_function=tokenizer_len
)

In [8]:
def extract_pages(file_path, lower_page_num, upper_page_num):

    """Function to extract text from PDF file"""

    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split(text_splitter=text_splitter)
    filtered_pages = [page for page in pages if page.metadata['page'] in list(range(lower_page_num, upper_page_num))]

    return filtered_pages

In [9]:
def format_chunk(text,
                 metadata,
                 chunk_number):

    """Function to format every text chunk for entry into vector database"""

    chunk = {}

    # PyPDF indexes pages starting from 0
    page_number = metadata['page'] + 1
    source = metadata['source']           
                     
    # For creating unique id of text chunk
    m = hashlib.md5()
    m.update(f"{source}-{chunk_number}".encode('utf-8'))
    uid = m.hexdigest()[:12]
    
    chunk['id'] = uid
    chunk['text'] = text
    chunk['metadata'] = {}
    chunk['metadata']['page_number'] = page_number
    chunk['metadata']['source'] = source
    
    return chunk

In [10]:
def create_dataset(pdf_files,
                   save_file_path,
                   ):

    """Function to extract text from PDF files, format for entry into vector database and save to disk."""
                       
    chunked_documents = []
                       
    for file in pdf_files:
        pages = extract_pages(file_path=file['path'],
                              lower_page_num=file['lower_page_num'],
                              upper_page_num=file['upper_page_num'])
        
        for i, page in enumerate(pages):
            chunk = format_chunk(text=page.page_content,
                                 metadata=page.metadata,
                                 chunk_number=i)
            chunked_documents.append(chunk)

    with open(save_file_path, 'w') as f:
        for doc in chunked_documents:
            f.write(json.dumps(doc) + '\n')  

In [11]:
rbc_files = [
    {"path": "data/RBC_2022_AR.pdf",
     "lower_page_num": 3,
     "upper_page_num": 126} ,
    {"path": "data/RBC_2022_AIF.pdf",
     "lower_page_num": 3,
     "upper_page_num": 26}    
]

lcl_files = [
    {"path": "data/LCL_2022_AR.pdf",
     "lower_page_num": 0,
     "upper_page_num": 88},
    {"path": "data/LCL_2022_AIF.pdf",
     "lower_page_num": 4,
     "upper_page_num": 33}    
]

In [12]:
RBC_DATA_PATH = "data/rbc_dataset.jsonl"
LCL_DATA_PATH = "data/lcl_dataset.jsonl"

In [13]:
# Create RBC dataset
create_dataset(rbc_files, RBC_DATA_PATH)

# Create Loblaw dataset
create_dataset(lcl_files, LCL_DATA_PATH)

## Creating a database of text embeddings

We are using Chroma, an open-source database purpose built for AI embedding vectors and operations. This allows us to build a prototype embedding database that can scale to a far greater number of source documents.

In [14]:
import chromadb
from chromadb.config import Settings

EMBEDDING_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
CROSS_ENCODER_MODEL_NAME = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
cross_encoder_model = CrossEncoder(CROSS_ENCODER_MODEL_NAME)

tokenizer = tiktoken.get_encoding("cl100k_base")

rbc_data = []
with open('data/rbc_dataset.jsonl', 'r') as f:
    for line in f:
        rbc_data.append(json.loads(line))

lcl_data = []
with open('data/lcl_dataset.jsonl', 'r') as f:
    for line in f:
        lcl_data.append(json.loads(line))

In [15]:
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory=".chromadb/"
))

rbc_collection = client.get_or_create_collection("RBC", metadata={"hnsw:space": "cosine"})
lcl_collection = client.get_or_create_collection("LCL", metadata={"hnsw:space": "cosine"})

batch_size = 128

def populate_db(db_collection,
                data,
                embedding_model,
                batch_size):

                        
    for i in range(0, len(data), batch_size):
        batch = data[i : i + batch_size]
    
        texts = []
        ids = []
        metadatas = []
        
        for doc in batch:
            
            doc_id = doc["id"]
            text = doc["text"]
            metadata = doc["metadata"]

            ids.append(doc_id)
            metadatas.append(metadata)
            texts.append(text)
        
        # Generate embeddings for the entire batch
        embeddings = embedding_model.encode(
            texts, convert_to_tensor=False
        ).tolist()
        
        # Store the batch to the database
        db_collection.add(
            documents=texts, embeddings=embeddings, metadatas=metadatas, ids=ids
        )

In [16]:
# Populate database with RBC documents
populate_db(rbc_collection, rbc_data, embedding_model, batch_size)
print(len(rbc_data))
print(rbc_collection.count())

839
839


In [17]:
# Populate database with Loblaw documents
populate_db(lcl_collection, lcl_data, embedding_model, batch_size)
print(len(lcl_data))
print(lcl_collection.count())

578
578


In [18]:
# Persist database to disk
client.persist()

True

## Base Functions for Skills

### Retrieval Functions

In [19]:
def retrieve_docs(query,
                  k,
                  collection):

    """Function to retrieve top K documents from the database based on similarity to the query."""
                      
    # Calculate the embedding for the query
    embedded_query = embedding_model.encode(query, convert_to_tensor=False).tolist()
                 
    # Retrieve documenents from the database
    output = collection.query(
        query_embeddings=[embedded_query],
        n_results=k
    )
    
    return output    


def retrieve_news(query,
                  k):

    """Retrieve K news articles from Google News"""

    google_news = GoogleNews(period='30d')
    google_news.enableException(True)

    results = []
    google_news.search(f"{query}".replace(' ', '+'))

    page_num = 1
    
    while len(results) < k:

        try:
            google_news.get_page(page_num)
            google_news_result = google_news.results()
    
            if len(google_news_result) == 0:
                break
            results.extend(google_news_result)
            page_num += 1
        except:
            break
        
    google_news.clear()

    return results

def unique_elements(input):

    seen = {}
    for item in input:
        if item.get("link") not in seen:
            seen[item.get("link")] = item

    return list(seen.values())

### Utility Functions

In [20]:
def rank_documents(sentence_pairs, num_results):

    """Rank the retrieved documents based on their relevance to user query"""

    rankings = cross_encoder_model.predict(sentence_pairs)
    top_ranked_idx = rankings.argsort()[-num_results:][::-1]

    return top_ranked_idx


def create_context(documents, token_limit):
    
    """Convert the results of the retrieved documents into a context for the LLM prompt."""   
    
    context = ""
    num_tokens = 0

    for doc in documents:
        new_num_tokens = num_tokens + len(tokenizer.encode(doc))
        if new_num_tokens <= token_limit:
            context += f"{doc}\n\n"
            num_tokens = new_num_tokens 
        else:
            break

    return context

### Querying Functions

In [21]:
def query_database(collection,
                   query,
                   num_retrieved_docs,
                   num_ranked_docs,
                   token_limit):

    # Retrieve documents from database
    retrieved_docs = retrieve_docs(query, num_retrieved_docs, collection)

    # Create sentence pairs for ranking
    documents = retrieved_docs['documents'][0]
    sentence_pairs = [[query, doc] for doc in documents]
                       
    # Get indices of top ranked documents
    top_ranked_idx = rank_documents(sentence_pairs, num_ranked_docs)

    # Create updated document set
    ranked_results = {}
    ranked_results['documents'] = [[retrieved_docs['documents'][0][i] for i in top_ranked_idx]]
    ranked_documents = [retrieved_docs['documents'][0][i] for i in top_ranked_idx]

    # Create the context
    context = create_context(ranked_documents, token_limit) 

    return context


def query_google_news(query,
                      num_retrieved_docs,
                      num_ranked_docs,
                      token_limit):

    # Retrive Google News results
    retrieved_news = retrieve_news(query, num_retrieved_docs)
    # Filter out duplicates
    retrieved_news = unique_elements(retrieved_news)
    
    # Create sentence pairs for ranking
    documents = []
    for result in retrieved_news:
        text = f"{result['title']} + {result['desc']}"
        documents.append(text)

    sentence_pairs = [[query, doc] for doc in documents]

    # Get indices of top ranked documents
    top_ranked_idx = rank_documents(sentence_pairs, num_ranked_docs)
                          
    # Create updated document set
    ranked_documents = [documents[i] for i in top_ranked_idx]

    # Create the context
    context = create_context(ranked_documents, token_limit)

    return context
    

## Agent Skill Functions

In [22]:
# Setting constants
NUM_RETRIEVED_DB_DOCS = 100
NUM_RETRIEVED_NEWS_DOCS = 20
NUM_RANKED_DOCS = 1
CONTEXT_TOKEN_LIMIT = 4000

def rbc_docs_skill(query):

    """Skill to create a context for the LLM by retrieving relevant documents from the RBC document database"""
    return query_database(collection=rbc_collection,
                          query=query,
                          num_retrieved_docs=NUM_RETRIEVED_DB_DOCS,
                          num_ranked_docs=NUM_RANKED_DOCS,
                          token_limit=CONTEXT_TOKEN_LIMIT)

def lcl_docs_skill(query):

    """Skill to create a context for the LLM by retrieving relevant documents from the Loblaw document database"""
    return query_database(collection=lcl_collection,
                          query=query,
                          num_retrieved_docs=NUM_RETRIEVED_DB_DOCS,
                          num_ranked_docs=NUM_RANKED_DOCS,
                          token_limit=CONTEXT_TOKEN_LIMIT)    

def google_news_skill(query):

    """Skill to create a context for the LLM by retrieving relevant news articles from Google News"""
    return query_google_news(query=query,
                             num_retrieved_docs=NUM_RETRIEVED_NEWS_DOCS,
                             num_ranked_docs=NUM_RANKED_DOCS,
                             token_limit=CONTEXT_TOKEN_LIMIT)


### Skill Testing Questions

In [23]:
print(rbc_docs_skill("company strategy"))

Strategic priorities 
OUR STRATEGY  PROGRESS IN 2022  PRIORITIES IN 2023  
Grow and deepen client relationships  Strengthened relationships with key clients,  
resulting in high quality mandates and notable  wins in Corporate & Investment Banking and  Global Markets, for example:  
• Financial Advisor to DigitalBridge and IFM  
Investors on US$11 billion landmark  communications infrastructure take-private  transaction  
 Financial Advisor and Lead Arranger to The  
Vertex Company and American Industrial  Partners on its merger with Vectrus, creating a  US$2.1 billion entity  Deliver holistic coverage to clients and drive  
multi-product client relationships to gain market  share  
Expand client coverage in underpenetrated  
sectors and products  
Lead with advice and extend capabilities  Expanded advisory and thought leadership  capabilities inclusive of Sustainable Finance
1 and  
Private Capital  
Refreshed our Global Markets strategy to deliver  
holistic client service providing 

In [24]:
print(lcl_docs_skill("company strategy"))

The Company’s inability to effectively predict market activity, leverage customer preferences and spending 
patterns and respond in a timely manner to trends, or compete effectively with its current or future competitors could result in, among other things, reduced market share and reduced profitability. If the Company is ineffective in responding to consumer trends or in executing its strategic plans, its financial performance could be adversely affected. The failure to effectively respond to customer trends may adversely impact the Company’s relationship with its customers. The Company closely monitors market developments and market share trends. Failure by the Company to sustain its competitive position could adversely affect the Company’s financial performance. 
Electronic Commerce and Disruptive Technologies The Company’s e-commerce strategy is a growing business




In [25]:
print(google_news_skill("loblaw company strategy"))

Loblaw's Retail Strategy Pays Off in Q3 + Canadian company delivers adjusted EBITDA growth of 10.3%




In [26]:
print(lcl_docs_skill("environmental priorities"))

Environmental,  
Social and  
Governance
Our purpose-led approach has strong relevance to our Environmental,  
Social and Governance (ESG) priorities, guiding us as we work to fight  
climate change and advance social equity.
Fighting Climate Change
Net-zero scope 1 and 2 by 2040, and scope 3 by 2050 
In 2020, we met our goal of a 30% reduction in our carbon 
footprint, ten years early. We have since extended our  
focus to net-zero, which includes eliminating our enterprise  
green-house gas (GHG) emissions by 2050. In 2022 we: 
• Completed 250 carbon reduction projects.
• Completed our climate risk assessment, evaluating our risk exposure and opportunities across multiple climate scenarios.
• Published our inaugural TCFD-aligned report. 
Tackling Plastic Waste 
Loblaw has a longstanding commitment to tackling plastic waste, 
working both at home and internationally on solutions. In 2022 we:




# Using GPT-4 For Skill Selection

## Implementing A Basic Control Strategy

In [27]:
controller_system_prompt = """
You are the control module for an expert Financial Analyst who is responsible 
for assisting human users with understanding and interpreting financial reports
and other sources of data about a cross-section of Canadian companies.
Your role is to determine which of your skills, called chains, are most likely 
to help the Financial Analyst with answering the user question.
"""

controller_prompt_template = Template("""
### Goal
Determine which of the following chains, formatted as a Python 
dictionary {chain_name: chain_description}, are relevant to the user message,
and reformulate the user message into a brief search query.

### Chains
$chains

### Instructions
- Generate a response for each of the provided chains
- In each response, reformulate the user message into a brief and relevant search query
- For chains that are irrelevant, respond with "N/A" for the reformulated query
- Rank your responses from most to least relevant
- Format each response precisely as: rank;chain_name;reformulated search query

### Examples
$few_shot_examples

############
User message:
$user_message

Response:
""")

chains = {
    "rbc_investor_material": "Retrieve information from Royal Bank of Canada (RBC)'s most recent Annual Report and Annual Information form, documents that contain important updates for investors about company performance and operations.", 
    "lcl_investor_material": "Retrieve information from Loblaw Companies Limited (Loblaw)'s most recent Annual Report and Annual Information form, documents that contain important updates for investors about company performance and operations.", 
    "google_news_search": "Search Google News for recent information related to the user message. Be sure to mention the relevant company name in the reformulated search query."
}

few_shot_examples = "\n***\n".join(["""
User message:
Does Loblaw operate any drug stores outside of its core grocery busniess?

Response:
1;lcl_investor_material;drug store brands
2;google_news_search;Loblaw-operated drug store brands
3;rbc_investor_material;N/A
""", 

"""
User message:
What rewards programs does RBC offer to banking customers?

Response:
1;rbc_investor_material;rewards programs
2;google_news_search;RBC rewards programs
3;lcl_investor_material;N/A
"""
])

In [28]:
# Preview the prompt after filling in variables

print(controller_prompt_template.substitute(
    chains=chains,
    few_shot_examples = few_shot_examples,
    user_message=''
))


### Goal
Determine which of the following chains, formatted as a Python 
dictionary {chain_name: chain_description}, are relevant to the user message,
and reformulate the user message into a brief search query.

### Chains
{'rbc_investor_material': "Retrieve information from Royal Bank of Canada (RBC)'s most recent Annual Report and Annual Information form, documents that contain important updates for investors about company performance and operations.", 'lcl_investor_material': "Retrieve information from Loblaw Companies Limited (Loblaw)'s most recent Annual Report and Annual Information form, documents that contain important updates for investors about company performance and operations.", 'google_news_search': 'Search Google News for recent information related to the user message. Be sure to mention the relevant company name in the reformulated search query.'}

### Instructions
- Generate a response for each of the provided chains
- In each response, reformulate the user message in

## Controller Test Run

In [29]:
# Use GPT to generate a response to the control request

user_message = "What are Loblaw's environmental priorities?"

message = controller_prompt_template.substitute(
    chains=chains,
    few_shot_examples = few_shot_examples,
    user_message=user_message,
)

response = get_completion(message, controller_system_prompt, model='gpt-4')
print(response)

1;lcl_investor_material;environmental priorities
2;google_news_search;Loblaw environmental priorities
3;rbc_investor_material;N/A


In [30]:
# Parse the response
def parse_top_response(response):
  try:
    responses = response.split('\n')
    rank, chain_name, query = responses[0].split(';')
    return chain_name, query
  except Exception as e:
    print("Failed to parse response: {response}")
    return None

# Map chain names to callable skill functions
skills_map = {
    "rbc_investor_material": rbc_docs_skill,
    "lcl_investor_material": lcl_docs_skill,
    "google_news_search": google_news_skill
}

In [31]:
# Check the parsed top response
skill, query = parse_top_response(response)
print(f"{skill}, {query}")

lcl_investor_material, environmental priorities


In [32]:
# Apply the top-ranked skill with the generated query
print(skills_map[skill](query))

Environmental,  
Social and  
Governance
Our purpose-led approach has strong relevance to our Environmental,  
Social and Governance (ESG) priorities, guiding us as we work to fight  
climate change and advance social equity.
Fighting Climate Change
Net-zero scope 1 and 2 by 2040, and scope 3 by 2050 
In 2020, we met our goal of a 30% reduction in our carbon 
footprint, ten years early. We have since extended our  
focus to net-zero, which includes eliminating our enterprise  
green-house gas (GHG) emissions by 2050. In 2022 we: 
• Completed 250 carbon reduction projects.
• Completed our climate risk assessment, evaluating our risk exposure and opportunities across multiple climate scenarios.
• Published our inaugural TCFD-aligned report. 
Tackling Plastic Waste 
Loblaw has a longstanding commitment to tackling plastic waste, 
working both at home and internationally on solutions. In 2022 we:




## Calling GPT With Context

In [33]:
analyst_system_prompt = """
You are an expert Financial Analyst who is responsible for assisting
human users with understanding and interpreting financial reports
and other sources of data about a cross-section of Canadian companies.
Relevant information to users' queries will be provided as input. If
it is useful, use this information to help answer the question.
"""

analyst_prompt_template = Template("""
### Goal
Respond to the user's message, using the following additional information, if it is useful.

### User Message
$user_message

### Context
$context

### Instructions
- Respond to the User Message to the best of your ability.
- Only provide responses that are factually correct.
- If you don't know the answer to a question, say this in your response.

Response:
""")

context = skills_map[skill](query)

In [34]:
# Check the prompt with supplied context
print(analyst_prompt_template.substitute(
    user_message=user_message,
    context=context
))


### Goal
Respond to the user's message, using the following additional information, if it is useful.

### User Message
What are Loblaw's environmental priorities?

### Context
Environmental,  
Social and  
Governance
Our purpose-led approach has strong relevance to our Environmental,  
Social and Governance (ESG) priorities, guiding us as we work to fight  
climate change and advance social equity.
Fighting Climate Change
Net-zero scope 1 and 2 by 2040, and scope 3 by 2050 
In 2020, we met our goal of a 30% reduction in our carbon 
footprint, ten years early. We have since extended our  
focus to net-zero, which includes eliminating our enterprise  
green-house gas (GHG) emissions by 2050. In 2022 we: 
• Completed 250 carbon reduction projects.
• Completed our climate risk assessment, evaluating our risk exposure and opportunities across multiple climate scenarios.
• Published our inaugural TCFD-aligned report. 
Tackling Plastic Waste 
Loblaw has a longstanding commitment to tackling 

## Analyst Test Run

In [35]:
message = analyst_prompt_template.substitute(
    user_message=user_message,
    context=context
)

response = get_completion(message, analyst_system_prompt, model='gpt-4')
pprint(response)

("Loblaw's environmental priorities include fighting climate change and "
 'tackling plastic waste. They aim to achieve net-zero scope 1 and 2 emissions '
 'by 2040, and scope 3 emissions by 2050. In 2020, Loblaw met its goal of a '
 '30% reduction in carbon footprint ten years early. They have completed 250 '
 'carbon reduction projects and published their inaugural TCFD-aligned report. '
 'Additionally, Loblaw is committed to addressing plastic waste and is '
 'actively working on solutions both domestically and internationally.')


In [36]:
response

"Loblaw's environmental priorities include fighting climate change and tackling plastic waste. They aim to achieve net-zero scope 1 and 2 emissions by 2040, and scope 3 emissions by 2050. In 2020, Loblaw met its goal of a 30% reduction in carbon footprint ten years early. They have completed 250 carbon reduction projects and published their inaugural TCFD-aligned report. Additionally, Loblaw is committed to addressing plastic waste and is actively working on solutions both domestically and internationally."

# Self-Assessment Via Fact-Checking

We have four steps for evaluating factuality of a statement:
1. Generate Knowledge base (KBs) search queries
2. Search and retrieve documents from KBs using generated queries
3. Rank and extract the most relevant passages from the retrieved documents
4. Generate a True/False response for the given statement and fact evidence passage.

System returns  `True` if we could find enough evidence in KBs to support the fact. Else `False`.

Let's continue with the same example user message and AI-generated output.

In [37]:
# Recall our working example
print("User Message:")
print(user_message, '\n')
print("AI Response:")
pprint(response)

User Message:
What are Loblaw's environmental priorities? 

AI Response:
("Loblaw's environmental priorities include fighting climate change and "
 'tackling plastic waste. They aim to achieve net-zero scope 1 and 2 emissions '
 'by 2040, and scope 3 emissions by 2050. In 2020, Loblaw met its goal of a '
 '30% reduction in carbon footprint ten years early. They have completed 250 '
 'carbon reduction projects and published their inaugural TCFD-aligned report. '
 'Additionally, Loblaw is committed to addressing plastic waste and is '
 'actively working on solutions both domestically and internationally.')


Our next goal is to validate whether we have supporting evidence for this response.

## Generate Search Queries for Fact Checking

In [38]:
NUM_QUERIES = 3 #number of search queries to generate
NUM_WIKI_ARTICLES = 2 #max number of wiki docs to scrape for each query
NUM_GOOGLE_NEWS_ARTICLES = 3 #max number of google news docs to scrape for each query
NUM_GOOGLE_SEARCH_ARTICLES = 3 #max number of google search docs to scrape for each query
DEFAULT_MODEL = "gpt-4"

NUM_RETRIEVED_DB_DOCS = 100
NUM_RETRIEVED_NEWS_DOCS = 20
NUM_RANKED_DOCS = 10
CONTEXT_TOKEN_LIMIT = 4000

In [39]:
fc_queries_system_prompt = """
You are the query generation mechanism for a fact-checking agent. 
Your role is to generate search queries that will be used to fetch 
articles that can help with verifying facts in an AI-generated response.
Your response should only be a list of queries separated by '\n'.
e.g.: `foo\n bar\n baz`
"""

fc_queries_prompt_template = Template("""
### Goal
Generate fact-checking queries for:

$ai_response

### Instructions
- Generate the response as a list of queries separated by '\n'.

### Examples
foo\n bar\n baz
query1\n query2\n query3

""")

In [40]:
def generate_queries(ai_response, num_queries=NUM_QUERIES, model="gpt-3.5-turbo", temprature=0):

    fc_message = fc_queries_prompt_template.substitute(ai_response=ai_response)        
    fc_queries_response = get_completion(fc_message, fc_queries_system_prompt)
    
    queries = fc_queries_response.strip().split("\n")[:num_queries]
    for i in range(len(queries)):
      queries[i] = queries[i].strip()
      if queries[i][:2] == "- ":
          queries[i] = queries[i][2:].strip()
    queries = [q for q in queries if q is not None and len(q)]
    return queries

In [41]:
queries = generate_queries(response, num_queries=NUM_QUERIES, model='gpt-4')
pprint(queries)

['Loblaw environmental priorities',
 'Loblaw climate change initiatives',
 'Loblaw plastic waste reduction']


## Use Queries to Search Available Knowledge Bases for Supporting Evidence

### Utility Functions

In [42]:
def chunk_text(text):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    
    def tokenizer_len(text):
        tokens = tokenizer.encode(text, max_length=512, truncation=False)
        return len(tokens) 

    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=128,
            chunk_overlap=64,
            length_function=tokenizer_len
    )
    return text_splitter.split_text(text)


def search_wiki(queries, n_max_articles=NUM_WIKI_ARTICLES):
    documents = []
    wiki_api_url = "https://en.wikipedia.org/w/api.php"
    for query in queries:
        #search documents
        search_response = requests.get(
            wiki_api_url, 
            params={
                    "action": "query",
                    "format": "json",
                    "list": "search",
                    "srsearch": query
                }
        ).json()
        page_titles = [q['title'] for q in search_response['query']['search'][:n_max_articles]]

        #fetch documents
        wiki_wiki = wikipediaapi.Wikipedia(
                    language='en',
                    extract_format=wikipediaapi.ExtractFormat.WIKI
        )
        for page_title in page_titles:
            try:
                document = wiki_wiki.page(page_title).text
                url = f"https://en.wikipedia.org/wiki/{page_title}"
            except:
                continue
            
            text_chunks = chunk_text(document)
            for chunk in text_chunks:
                documents.append({
                    "text": chunk,
                    "metadata": {"source": url},
                })
    return documents

def remove_duplicate_docs(fact_documents):
    texts = set()   
    unique_fact_documents = []
    for doc in fact_documents:
        if doc['text'] in texts:
            continue
        unique_fact_documents.append(doc)
        texts.add(doc['text'])
    return unique_fact_documents

def search_google(queries, n=NUM_GOOGLE_SEARCH_ARTICLES):
    api_key = os.getenv("GOOGLE_API_KEY")
    cx = os.getenv("CX")
    google_search_obj = build("customsearch", "v1", developerKey=api_key)
    search_docs = []
    for query in queries:
        response =  google_search_obj.cse().list(q=query, cx=cx, num=n).execute() #load from env
        data = response.get("items", [])
        for d in data[:n]:
            search_docs.append({
                "text": f"{d['title']}\n{d['snippet']}",
                "metadata": {"source": d['link']},
            })   
    return search_docs  

def retrieve_fact_docs(
        queries, 
        n_news_docs=NUM_GOOGLE_NEWS_ARTICLES, 
        n_wiki_docs=NUM_WIKI_ARTICLES,
        n_google_search_docs=NUM_GOOGLE_SEARCH_ARTICLES):
    #Google news docs
    news_docs = []
    for query in queries:
        try:
            news_articles = retrieve_news(query, n_news_docs)
        except:
            continue
        for article in news_articles:
            news_docs.append({
                "text": f"{article['title']}\n{article['desc']}",
                "metadata": {"source": article['link']},
            })

    #Wiki docs
    wiki_docs = search_wiki(queries, n_wiki_docs)

    #Google search docs
    google_search_docs = search_google(queries, n_google_search_docs)

    # Combine all docs and remove duplicates
    fact_documents = news_docs + wiki_docs + google_search_docs
    fact_documents = [{'id': str(i), **doc} for i, doc in enumerate(fact_documents)]
    fact_documents = remove_duplicate_docs(fact_documents)

    return fact_documents


## Retrieve Documents Related to Generated Queries

In [43]:
fact_documents = retrieve_fact_docs(queries)

In [44]:
# Print an example
pprint(fact_documents[0])

{'id': '0',
 'metadata': {'source': 'https://ca.finance.yahoo.com/news/loblaw-esg-report-reveals-8-180000904.html'},
 'text': 'Loblaw ESG Report reveals 8% reduction in greenhouse gas (GHG) '
         'emissions, among many other achievements designed to fight climate '
         'change and advance social equity\n'
         'Today, Loblaw Companies Limited (TSX: L) ("Loblaw") released its '
         '2022 Environmental, Social and Governance (ESG) report. The '
         "company's 16th annual report..."}


## Find The Most Related Search Results

- Extract the most-informative sections from the retrieved documents
- Use the sentence transformer to encode chunks of each document as overlapping chunk vectors. Store them in a dictionary called `vector_dict`
- Encode the `response` using same transformer, call this the "query" vector
- Find the search result chunks with the highest cosine similarity to the previously generated response

In [45]:
def extract_most_relevant_fact_passage(test_response, fact_documents, max_passage_tokens=300):
    fact_evidence_collection = client.get_or_create_collection(f"FEC", metadata={"hnsw:space": "cosine"})
    populate_db(fact_evidence_collection, fact_documents, embedding_model, batch_size)

    relevant_fact_passage = query_database(
        collection=fact_evidence_collection,
        query=test_response,
        num_retrieved_docs=5,
        num_ranked_docs=NUM_RANKED_DOCS,
        token_limit=CONTEXT_TOKEN_LIMIT
    )
    relevant_fact_passage = tokenizer.decode(tokenizer.encode(relevant_fact_passage)[:max_passage_tokens])
    client.delete_collection("FEC")
    return relevant_fact_passage

In [46]:
relevant_fact_passages = extract_most_relevant_fact_passage(response, fact_documents)

In [47]:
print(relevant_fact_passages)

Loblaw ESG Report reveals 8% reduction in greenhouse gas (GHG ...
May 3, 2023 ... ... Companies Limited (TSX: L) ("Loblaw") released its 2022 Environmental, ... including its two priority areas: fighting climate change and ...

Loblaw issues 2021 ESG Report, outlines plans to fight climate ...
May 5, 2022 ... Loblaw issues 2021 ESG Report, outlines plans to fight climate change and advance social equity · Achieving net-zero emissions by 2040 for Scope ...

Loblaw issues 2021 ESG Report, outlines plans to fight climate ...
May 5, 2022 ... ... “Loblaw”) released its 2021 environmental, social, and governance ... new commitments focused on two priority areas: fighting climate ...

Loblaw takes action on climate change with net-zero commitment
Mar 23, 2022 ... In 2016, Loblaw committed to a 30-per-cent reduction in corporate carbon emissions by 2030. The company met that target in 2020, due to its ...

Loblaw eyes net-zero emissions for operations by 2040 ...
Mar 24, 2022 ... Loblaw said i

## Use an Evaluator Agent to Perform Fact Checking

Using the content retrieved via search, determine whether it has contains information or evidence that supports the initial AI-generated response.

In [48]:
fc_system_prompt = "You are a fact evaluator Agent responsible for fact checking AI generated statements with the help of a given related information passage."

fc_message_prompt_template = Template("""
### Goal
Evaluate the factuality of an AI-Generated Response to a User Message using the given Retrieved Context and Related Information.

### Instructions
- First, write out in a step-by-step manner your reasoning to be sure that your conclusion is correct. Avoid simply stating the correct answer at the outset. 
- Then, on a new line, write "True" if the Related Information supports the AI-Generated Response, otherwise write "False".
                    
### Example Output
There was sufficient Related Information to support the claim.

True

************************

There was not sufficient information to support the claim.

False
                                      
### User Message
$user_message                                      

### AI-Generated Response
$ai_response

### Retrieved Context
$context

### Related Information
$passage

### Response
""")

In [49]:
def evaluate_fact(user_message, context, relevant_fact_passage, ai_response):
    message = fc_message_prompt_template.substitute(user_message=user_message, context=context, ai_response=ai_response, passage=relevant_fact_passage)        
    fc_compare_response = get_completion(message, fc_system_prompt)
    return fc_compare_response.strip()

In [50]:
response

"Loblaw's environmental priorities include fighting climate change and tackling plastic waste. They aim to achieve net-zero scope 1 and 2 emissions by 2040, and scope 3 emissions by 2050. In 2020, Loblaw met its goal of a 30% reduction in carbon footprint ten years early. They have completed 250 carbon reduction projects and published their inaugural TCFD-aligned report. Additionally, Loblaw is committed to addressing plastic waste and is actively working on solutions both domestically and internationally."

In [51]:
context

'Environmental,  \nSocial and  \nGovernance\nOur purpose-led approach has strong relevance to our Environmental,  \nSocial and Governance (ESG) priorities, guiding us as we work to fight  \nclimate change and advance social equity.\nFighting Climate Change\nNet-zero scope 1 and 2 by 2040, and scope 3 by 2050 \nIn 2020, we met our goal of a 30% reduction in our carbon \nfootprint, ten years early. We have since extended our  \nfocus to net-zero, which includes eliminating our enterprise  \ngreen-house gas (GHG) emissions by 2050. In 2022 we: \n• Completed 250 carbon reduction projects.\n• Completed our climate risk assessment, evaluating our risk exposure and opportunities across multiple climate scenarios.\n• Published our inaugural TCFD-aligned report. \nTackling Plastic Waste \nLoblaw has a longstanding commitment to tackling plastic waste, \nworking both at home and internationally on solutions. In 2022 we:\n\n'

In [52]:
relevant_fact_passages

'Loblaw ESG Report reveals 8% reduction in greenhouse gas (GHG ...\nMay 3, 2023 ... ... Companies Limited (TSX: L) ("Loblaw") released its 2022 Environmental, ... including its two priority areas: fighting climate change and\xa0...\n\nLoblaw issues 2021 ESG Report, outlines plans to fight climate ...\nMay 5, 2022 ... Loblaw issues 2021 ESG Report, outlines plans to fight climate change and advance social equity · Achieving net-zero emissions by 2040 for Scope\xa0...\n\nLoblaw issues 2021 ESG Report, outlines plans to fight climate ...\nMay 5, 2022 ... ... “Loblaw”) released its 2021 environmental, social, and governance ... new commitments focused on two priority areas: fighting climate\xa0...\n\nLoblaw takes action on climate change with net-zero commitment\nMar 23, 2022 ... In 2016, Loblaw committed to a 30-per-cent reduction in corporate carbon emissions by 2030. The company met that target in 2020, due to its\xa0...\n\nLoblaw eyes net-zero emissions for operations by 2040 ...\nMar 

In [53]:
print(evaluate_fact(user_message, context, relevant_fact_passages, response))

1. The user asked about Loblaw's environmental priorities.
2. The AI-generated response mentioned fighting climate change and tackling plastic waste as Loblaw's environmental priorities.
3. The Retrieved Context confirms that Loblaw's priorities are fighting climate change and advancing social equity.
4. The AI-generated response provided details about Loblaw's goals for net-zero emissions, which are supported by the Related Information.
5. The AI-generated response mentioned Loblaw's commitment to addressing plastic waste, which is also supported by the Retrieved Context.

There was sufficient Related Information to support the claim.

True


## Fact-Checking Test Run

In [54]:
def run_fact_check(user_message, context, ai_response):
  search_queries = generate_queries(ai_response, NUM_QUERIES)
  document_chunks = retrieve_fact_docs(search_queries)
  relevant_fact_passage = extract_most_relevant_fact_passage(ai_response, fact_documents)
  return evaluate_fact(user_message, context, relevant_fact_passage, ai_response)

In [55]:
response

"Loblaw's environmental priorities include fighting climate change and tackling plastic waste. They aim to achieve net-zero scope 1 and 2 emissions by 2040, and scope 3 emissions by 2050. In 2020, Loblaw met its goal of a 30% reduction in carbon footprint ten years early. They have completed 250 carbon reduction projects and published their inaugural TCFD-aligned report. Additionally, Loblaw is committed to addressing plastic waste and is actively working on solutions both domestically and internationally."

In [56]:
fact_checking_response = run_fact_check(user_message, context, response)

In [57]:
print(fact_checking_response)

1. The user asked about Loblaw's environmental priorities.
2. The AI-generated response mentioned fighting climate change and tackling plastic waste as Loblaw's environmental priorities.
3. The Retrieved Context confirms that Loblaw's priorities are fighting climate change and advancing social equity.
4. The AI-generated response provided details about Loblaw's goals for net-zero emissions, which are supported by the Retrieved Context.
5. The AI-generated response mentioned Loblaw's commitment to tackling plastic waste, which is also supported by the Retrieved Context.

There was sufficient Related Information to support the claim.

True


# Building An End-To-End Assistant

## Collecting All Steps

In [58]:
### Bring everything together so that we can build a single interface for the end user, so we can go directly from
### user message to a fact-checked response.

def financial_analyst_chain(user_message, verbose=False):

  # The User's message to the AI system
  if verbose:
    print(f"User's Message:\n{user_message}\n")

  # Apply the Controller prompt template to build the Controller message
  controller_message = controller_prompt_template.substitute(
      chains=chains,
      few_shot_examples = few_shot_examples,
      user_message=user_message,
  )

  # Get the Controller's response
  controller_response = get_completion(controller_message, controller_system_prompt, model='gpt-4')
  if verbose:
    print(f"Controller's Response:\n{controller_response}\n")

  # Pasre the Controller's response
  skill, query = parse_top_response(controller_response)
  if verbose:
    print(f"Parsed Controller Response:\nskill: {skill}, query: {query}\n")

  # Use the selected skill to get relevant contextual information
  context = skills_map[skill](query)
  if verbose:
    print("Retrieved Contextual Information")
    pprint(context)
    print()

  # Apply the Analyst prompt template to build the Analyst message
  analyst_message = analyst_prompt_template.substitute(
      user_message=user_message,
      context=context
  )

  # Get the Analyst's response 
  analyst_response = get_completion(analyst_message, analyst_system_prompt, model='gpt-4')
  if verbose:  
    pprint(f"Analyst's Response:\n{analyst_response}")
    print()

  # Run Fact-Checking and get the response
  fact_checking_response = run_fact_check(user_message, context, analyst_response)
  if verbose:
    print(f"Fact-Checking Result:\n{fact_checking_response}\n")

  return {
      "user_message": user_message,
      "controller_response": controller_response,
      "selected_skill": skill,
      "generated_query": query,
      "retrieved_context": context,
      "analyst_response": analyst_response,
      "fact_checking_response": fact_checking_response,
      "fact_check_passed": True if fact_checking_response.split()[-1] == 'True' else False
  }

In [59]:
user_message = "What are Loblaw's environmental priorities?"
analyst_output = financial_analyst_chain(user_message, verbose=True)

User's Message:
What are Loblaw's environmental priorities?

Controller's Response:
1;lcl_investor_material;environmental priorities
2;google_news_search;Loblaw environmental priorities
3;rbc_investor_material;N/A

Parsed Controller Response:
skill: lcl_investor_material, query: environmental priorities

Retrieved Contextual Information
('Environmental,  \n'
 'Social and  \n'
 'Governance\n'
 'Our purpose-led approach has strong relevance to our Environmental,  \n'
 'Social and Governance (ESG) priorities, guiding us as we work to fight  \n'
 'climate change and advance social equity.\n'
 'Fighting Climate Change\n'
 'Net-zero scope 1 and 2 by 2040, and scope 3 by 2050 \n'
 'In 2020, we met our goal of a 30% reduction in our carbon \n'
 'footprint, ten years early. We have since extended our  \n'
 'focus to net-zero, which includes eliminating our enterprise  \n'
 'green-house gas (GHG) emissions by 2050. In 2022 we: \n'
 '• Completed 250 carbon reduction projects.\n'
 '• Completed our

In [60]:
def self_assessed_analyst(user_message, verbose=False):
  analyst_output = financial_analyst_chain(user_message, verbose=verbose)
  print(f"{analyst_output['user_message']}\n")
  if analyst_output['fact_check_passed']:
    print("NOTICE: While automated fact-checking was successful, the factual accuracy of AI-generated responses is not guaranteed.\n")
    pprint(analyst_output['analyst_response'])
  else:
    print("WARNING: Supporting evidence was not found, please interpret with caution.\n")
    pprint(analyst_output['analyst_response'])

## Testing with additional user messages

In [61]:
self_assessed_analyst("What are Loblaw's environmental priorities?", verbose=False)

What are Loblaw's environmental priorities?

NOTICE: While automated fact-checking was successful, the factual accuracy of AI-generated responses is not guaranteed.

("Loblaw's environmental priorities are focused on fighting climate change and "
 'tackling plastic waste. They have set goals to achieve net-zero greenhouse '
 'gas emissions for scope 1 and 2 by 2040, and scope 3 by 2050. They also aim '
 'to ensure all of their control brand and in-store plastic packaging is '
 'either reusable or recyclable by 2025. Additionally, Loblaw is committed to '
 'sending zero food waste to landfill by the end of 2030. In recent years, '
 'they have completed numerous carbon reduction projects and published a '
 'climate risk assessment to further their environmental efforts.')


In [62]:
self_assessed_analyst("What does RBC have to say about the housing market?", verbose=False)

What does RBC have to say about the housing market?

NOTICE: While automated fact-checking was successful, the factual accuracy of AI-generated responses is not guaranteed.

("RBC's outlook on the housing market indicates that housing markets in Canada "
 'have already pulled back significantly since spring due to increases in '
 'interest rates. They expect moderate recessions in the U.S. and Canada in '
 'the first half of calendar 2023, with consumer spending contracting as '
 'higher borrowing costs and prices reduce household purchasing power. The '
 'economy is expected to grow again in the second half of calendar 2023, with '
 'slowing inflation allowing central banks to pause interest rate hikes in the '
 'coming months. RBC also highlights risks related to Canadian housing and '
 'household indebtedness in the current rising interest rate environment, '
 'which could lead to higher credit losses and impact housing market activity '
 'and prices.')


In [63]:
self_assessed_analyst("What does my uncle Bob have to say about the housing market?", verbose=False)

What does my uncle Bob have to say about the housing market?


("I'm sorry, but I don't have any information about your uncle Bob's opinion "
 'on the housing market. However, if you have any questions about the housing '
 "market or financial analysis, I'd be happy to help.")


In [64]:
self_assessed_analyst("In which business area has Loblaw been most successful?", verbose=False)

In which business area has Loblaw been most successful?

NOTICE: While automated fact-checking was successful, the factual accuracy of AI-generated responses is not guaranteed.

('Loblaw has been most successful in its retail excellence, particularly in '
 'food and drug retail. In recent financial reports, the company achieved '
 'same-store sales growth of +4.7% in food retail and +6.9% in drug retail, '
 'with a total revenue of $56.5 billion, growing +6.3%. Their focus on '
 'strategic procurement, customer loyalty programs, and leveraging their scale '
 'and strategic assets has contributed to their success in these areas.')


In [65]:
self_assessed_analyst("In which business area has Loblaw been least successful?", verbose=False)

In which business area has Loblaw been least successful?


('Based on the provided information, it is not possible to definitively '
 'determine the least successful business area for Loblaw. However, it is '
 'mentioned that the company faces challenges in change management, process '
 'and efficiency, IT systems implementations, and data management. These '
 "challenges could potentially impact the company's operations and financial "
 'performance. Additionally, the company faces competitive pressures in its '
 'strategic growth areas of Everyday Digital Retail, Payments and Rewards, and '
 'Connected Healthcare. Failure to achieve success in these areas could '
 "adversely affect Loblaw's financial position and competitiveness.")


# Comparing with Vanilla GPT-4 Response

In [66]:
vanilla_response = """Loblaw Companies Limited, a Canadian food retailer, has outlined several environmental priorities as part of its corporate social responsibility strategy. Some of these priorities include:

1. Reducing greenhouse gas emissions: Loblaw is committed to reducing its carbon footprint by improving energy efficiency, investing in renewable energy, and reducing refrigerant emissions in its stores and distribution centers.

2. Sustainable sourcing: Loblaw aims to source products responsibly by working with suppliers to ensure sustainable practices, such as sourcing seafood from certified sustainable fisheries and promoting the use of certified sustainable palm oil.

3. Reducing waste: Loblaw is focused on reducing waste across its operations, including food waste, packaging waste, and single-use plastics. The company has set targets to reduce food waste and increase the recyclability of its packaging materials.

4. Energy efficiency: Loblaw is investing in energy-efficient technologies and practices to reduce energy consumption in its stores, distribution centers, and offices.

5. Water stewardship: Loblaw is working to reduce water usage in its operations and promote water conservation practices among its suppliers.

6. Biodiversity and habitat protection: Loblaw supports initiatives that protect and restore natural habitats and promote biodiversity, such as partnering with organizations like the World Wildlife Fund (WWF) to protect critical ecosystems.

7. Climate change adaptation: Loblaw is working to understand and address the potential impacts of climate change on its operations, supply chain, and customers, and is taking steps to build resilience to these impacts.

By focusing on these environmental priorities, Loblaw aims to minimize its environmental impact and contribute to a more sustainable future."""

message = "What are Loblaw's environmental priorities?"
fact_checking_response = run_fact_check(message, context='', ai_response=vanilla_response)
print(fact_checking_response)

1. The AI-generated response mentions that Loblaw is committed to reducing greenhouse gas emissions. The Related Information states that Loblaw committed to a 30% reduction in corporate carbon emissions by 2030 and met that target in 2020.

2. The response talks about sustainable sourcing, but the Related Information does not provide any details on this topic.

3. The response mentions reducing waste, including food waste and packaging waste. The Related Information confirms that Loblaw has made strides in food waste reduction with Flashfood and has moved to cut down on plastic waste.

4. The response discusses energy efficiency, but the Related Information does not provide any details on this topic.

5. The response talks about water stewardship, but the Related Information does not provide any details on this topic.

6. The response mentions biodiversity and habitat protection, but the Related Information does not provide any details on this topic.

7. The response discusses climate 